Universidad del Valle de Guatemala

Security Data Science

Augusto Alonso

Douglas de León - 18037

# Laboratorio 1: Phishing Detection

In [11]:
import pandas as pd
import features_utils as fu

In [12]:
dataset = pd.read_csv("./dataset_pishing.csv")
dataset.head()

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,0,1,0,0,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,1,0,0,0,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,https://support-appleld.com.secureupdate.duila...,1,0,1,0,0,0,0,0,1,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,http://rgipt.ac.in,0,0,0,0,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,0,1,0,0,0,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,legitimate


In [13]:
dataset.columns.values

array(['url', 'ip', 'nb_www', 'nb_com', 'nb_dslash', 'http_in_path',
       'punycode', 'port', 'tld_in_path', 'tld_in_subdomain',
       'abnormal_subdomain', 'nb_subdomains', 'prefix_suffix',
       'random_domain', 'shortening_service', 'path_extension',
       'nb_redirection', 'nb_external_redirection', 'length_words_raw',
       'char_repeat', 'shortest_words_raw', 'shortest_word_host',
       'shortest_word_path', 'longest_words_raw', 'longest_word_host',
       'longest_word_path', 'avg_words_raw', 'avg_word_host',
       'avg_word_path', 'phish_hints', 'domain_in_brand',
       'brand_in_subdomain', 'brand_in_path', 'suspecious_tld',
       'statistical_report', 'nb_hyperlinks', 'ratio_intHyperlinks',
       'ratio_extHyperlinks', 'ratio_nullHyperlinks', 'nb_extCSS',
       'ratio_intRedirection', 'ratio_extRedirection', 'ratio_intErrors',
       'ratio_extErrors', 'login_form', 'external_favicon',
       'links_in_tags', 'submit_email', 'ratio_intMedia',
       'ratio_extMedi

In [14]:
dataset["status"].value_counts()

legitimate    5715
phishing      5715
Name: status, dtype: int64

El dataset se encuentra balanceado en la columna objetivo (status).

## Feature generation

In [15]:
dataset['f1'] = dataset['url'].map(fu.f1) # Full URL length.
dataset['f2'] = dataset['url'].map(fu.f2) # Hostname length.
dataset['f4'] = dataset['url'].map(fu.f4) # Number of occurences of character '.' in URL.
dataset['f5'] = dataset['url'].map(fu.f5) # Number of occurences of character '-' in URL.
dataset['f6'] = dataset['url'].map(fu.f6) # Number of occurences of character '@' in URL.
dataset['f7'] = dataset['url'].map(fu.f7) # Number of occurences of character '?' in URL.
dataset['f8'] = dataset['url'].map(fu.f8) # Number of occurences of character '&' in URL.
dataset['f9'] = dataset['url'].map(fu.f9) # Number of occurences of character '|' in URL.
dataset['f10'] = dataset['url'].map(fu.f10) # Number of occurences of character '=' in URL.
dataset['f11'] = dataset['url'].map(fu.f11) # Number of occurences of character '_' in URL.
dataset['f12'] = dataset['url'].map(fu.f12) # Number of occurences of character '~' in URL.
dataset['f13'] = dataset['url'].map(fu.f13) # Number of occurences of character '%' in URL.
dataset['f14'] = dataset['url'].map(fu.f14) # Number of occurences of character '/' in URL.
dataset['f15'] = dataset['url'].map(fu.f15) # Number of occurences of character '*' in URL.
dataset['f16'] = dataset['url'].map(fu.f16) # Number of occurences of character ':' in URL.
dataset['f17'] = dataset['url'].map(fu.f17) # Number of occurences of character ',' in URL.
dataset['f18'] = dataset['url'].map(fu.f18) # Number of occurences of character ';' in URL.
dataset['f19'] = dataset['url'].map(fu.f19) # Number of occurences of character '$' in URL.
dataset['f20'] = dataset['url'].map(fu.f20) # Number of occurences of character '%20' or space in URL.
dataset['f25'] = dataset['url'].map(fu.f25) # Checks for HTTPS token in URL.
dataset['f26'] = dataset['url'].map(fu.f26) # Ratio of digits in full URL.
dataset['f27'] = dataset['url'].map(fu.f27) # Ratio of digits in hostname.

In [16]:
dataset.head()

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,f14,f15,f16,f17,f18,f19,f20,f25,f26,f27
0,http://www.crestonwood.com/router.php,0,1,0,0,0,0,0,0,0,...,3,0,1,0,0,0,0,False,0.000000,0.0
1,http://shadetreetechnology.com/V4/validation/a...,1,0,0,0,0,0,0,0,0,...,5,0,1,0,0,0,0,False,0.220779,0.0
2,https://support-appleld.com.secureupdate.duila...,1,0,1,0,0,0,0,0,1,...,5,0,1,0,0,0,0,True,0.150794,0.0
3,http://rgipt.ac.in,0,0,0,0,0,0,0,0,0,...,2,0,1,0,0,0,0,False,0.000000,0.0
4,http://www.iracing.com/tracks/gateway-motorspo...,0,1,0,0,0,0,0,0,0,...,5,0,1,0,0,0,0,False,0.000000,0.0
